# LangChain Expression Language (LCEL)

In this notebook, we will give a high level introduction to what **LangChain Expression Language (LCEL)** is and show we can use it with an MLX locally-deployed model.

## Notebook Setup
Throughout this notebook, we will largely be making use of LangChain alongside MLX. In order to do a direct comparison with how MLX works within LangChain, we will also provide an example using the standard OpenAI API.

In [1]:
# Importing the necessary Python libraries
import yaml
from typing import Any, Dict, List, Optional
from langchain_core.language_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.pydantic_v1 import root_validator, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.outputs import ChatGeneration, ChatResult
from mlx_lm import load as mlx_load
from mlx_lm import generate as mlx_generate

# Importing legacy LangChain things for demonstration purposes
from langchain.chains import ConversationChain

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setting constant values to represent model name and directory
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
BASE_DIRECTORY = '../models'
MLX_DIRECTORY = f'{BASE_DIRECTORY}/mlx'
mlx_model_directory = f'{MLX_DIRECTORY}/{MODEL_NAME}'

In [3]:
# Loading my personal OpenAI API key (NOT pushed up to GitHub)
with open('../sensitive/api-keys.yaml') as f:
    API_KEYS = yaml.safe_load(f)

## Chaining the Old Way
In the cell below, we will demonstrate the former way of chaining a prompt template together with an LLM. LangChain has offered multiple ways to chain prompts to LLMs, but perhaps one of the most popular ways was using the `ConversationChain`. Let's go ahead and set up a simple prompt template and chain that to the OpenAI API to demonstrate how this legacy option worked.

In [4]:
# Instantiating the OpenAI LLM
llm = ChatOpenAI(openai_api_key = API_KEYS['OPENAI_API_KEY'])

In [5]:
# Setting up the Chat prompt template
system_message_prompt = SystemMessagePromptTemplate.from_template(template = 'You are a helpful assistant.')
human_message_prompt = HumanMessagePromptTemplate.from_template(template = "History: {history}\nHuman: {input}")
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [6]:
# Instantiating the legacy conversation chain
legacy_conversation_chain = ConversationChain(llm = llm, prompt = chat_prompt)

In [7]:
# Making a call to the OpenAI API
response = legacy_conversation_chain.predict(input = 'Write me a haiku about flowers.')

print(response)

Petals soft and bright,
Dancing in the gentle breeze,
Nature's sweet delight.


## Why LCEL?

As you can see, the legacy syntax has historically not been the easiest to learn. In addition to the `ConversationChain` object, LangChain has provided many other objects that perform more or less the same thing, introducing confusion on when to use what.

Additionally, what is not particularly clear by using the `ConversationChain` object, but there is a simple memory that is being kept each time that object is called. This may or may not be preferable given your use case, but in any regard, it is not ideal that this is abstracted away.

LCEL attempts to simplify the process by allowing users to chain LangChain objects together directly using the pipe (`|`) delimiter. There are many benefits to using LCEL, and you can read more about them [at this page](https://python.langchain.com/docs/expression_language/).

In [8]:
# Instantiating the OpenAI LLM
llm = ChatOpenAI(openai_api_key = API_KEYS['OPENAI_API_KEY'])

# Setting up the Chat prompt template
chat_prompt = ChatPromptTemplate.from_template('{input}')

In [9]:
# Instantiating the new chain with the LCEL syntax
new_conversation_chain = chat_prompt | llm

In [10]:
# Invoking the model appropriately
response = new_conversation_chain.invoke({'input': 'Write me a haiku about Jar Jar Binks.'})

print(response.content)

Clumsy Gungan friend
Jar Jar brings laughter and joy
In a galaxy


# Using MLX with LCEL
By default, there is currently no mechanism within the LangChain libraries that supports MLX; however, we still can make use of MLX in a custom capacity. In the next few cells, we will demonstrate how we can do this appropriately.

In [11]:
# Instantiating the class representing the MLX Chat Model, inheriting from LangChain's BaseChatModel
class MLXChatModel(BaseChatModel):
    mlx_path: str
    mlx_model: Any = Field(default = None, exclude = True)
    mlx_tokenizer: Any = Field(default = None, exclude = True)
    max_tokens: int = Field(default = 1000)

    @property
    def _llm_type(self) -> str:
        return 'MLXChatModel'
    


    @root_validator()
    def load_model(cls, values: Dict) -> Dict:

        # Loading the model and tokenizer with the input string
        model, tokenizer = mlx_load(path_or_hf_repo = values['mlx_path'])
        
        # Saving the variables back appropriately
        values['mlx_model'] = model
        values['mlx_tokenizer'] = tokenizer
        return values
    


    def _generate(self, messages: List[BaseMessage], stop: Optional[List[str]]) -> ChatResult:

        # Instantiating an empty string to represent the prompt we will be generating in the end
        prompt = ''

        # Extracting the raw text from each of the LangChain message types
        for message in messages:
            prompt += f'\n\n{message.content}'

        # Generating the LLM response using MLX
        mlx_response = mlx_generate(
            model = self.mlx_model,
            tokenizer = self.mlx_tokenizer,
            max_tokens = self.max_tokens,
            prompt = prompt
        )

        # Returning the MLX response as a proper LangChain ChatResult object
        return ChatResult(generations = [ChatGeneration(message = AIMessage(content = mlx_response))])

In [12]:
# Instantiating the MLX model using our quantized Mistral 7B
mlx_model = MLXChatModel(mlx_path = mlx_model_directory)

In [13]:
# Setting up the Chat prompt template
system_message_prompt = SystemMessagePromptTemplate.from_template(template = 'You are a helpful assistant.')
human_message_prompt = HumanMessagePromptTemplate.from_template(template = "{input}")
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [14]:
# Instantiating the MLX chain with the LCEL syntax
mlx_chain = chat_prompt | mlx_model

In [15]:
# Generating the response with MLX + LCEL
response = mlx_chain.invoke(input = {'input': 'Write a haiku about Jar Jar Binks.'})

print(response.content)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Giddy, clumsy Gungan,
Bouncing through Star Wars' world,
Laughter, then disdain.


## Putting It All Together
Now that we've demonstrated how we can use LangChain to very rapidly swap out various models so that we can do quick testing. For our test below, I'm going to use 3 models: ChatGPT as provided by OpenAI, Mixtral as provided by Perplexity, and my local Mistral provided by MLX.

In [16]:
# Setting up the Chat prompt template
system_message_prompt = SystemMessagePromptTemplate.from_template(template = 'You are a helpful assistant.')
human_message_prompt = HumanMessagePromptTemplate.from_template(template = "{input}")
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Setting an input prompt
input_prompt = {'input': 'Write a haiku about Jar Jar Binks.'}

In [17]:
# Iterating over each of our 3 models
for model_type in ['OpenAI', 'Perplexity (Mixtral)', 'MLX (Mistral)']:
    
    print(f'Currently testing the following model: {model_type}')

    # Loading the respective model
    if model_type == 'OpenAI':
        model = ChatOpenAI(openai_api_key = API_KEYS['OPENAI_API_KEY'])
    elif model_type == 'Perplexity (Mixtral)':
        model = ChatOpenAI(api_key = API_KEYS['PERPLEXITY_API_KEY'],
                           base_url = 'https://api.perplexity.ai',
                           model = 'mixtral-8x7b-instruct')
    elif model_type == 'MLX (Mistral)':
        model = MLXChatModel(mlx_path = mlx_model_directory)

    # Instantiating the chain with the LCEL syntax
    llm_chain = chat_prompt | model

    # Invoking the chain
    response = llm_chain.invoke(input = input_prompt)
    print(response.content)
    print()

Currently testing the following model: OpenAI
Mesa clumsy friend,
Jar Jar dances through the swamp,
Gungans sing his name.

Currently testing the following model: Perplexity (Mixtral)
Gungan in a bind,
Clumsy words, but heart kind,
Jar Jar, hard to hate.

Currently testing the following model: MLX (Mistral)


Giddy, clumsy Gungan,
Bouncing through Star Wars' world,
Laughter, then disdain.



In [71]:
import json

class MLXModelParameters():

    def __init__(self, temp = 0.7, max_tokens = 1000):
        self.temp = temp
        self.max_tokens = max_tokens

    def __str__(self):
        return f'Temperature: {self.temp}\nMax Tokens: {self.max_tokens}'
    
    def __repr__(self):
        return f'Temperature: {self.temp}\nMax Tokens: {self.max_tokens}'

    def update_temp(self, new_temp):
        self.temp = new_temp

    def update_max_tokens(self, new_max_tokens):
        self.max_tokens = new_max_tokens

    def to_json(self):
        return { 'temp': self.temp, 'max_tokens': self.max_tokens }

In [72]:
model_params = MLXModelParameters()

In [73]:
model_params.update_temp(new_temp = 1)

In [74]:
print(model_params)

Temperature: 1
Max Tokens: 1000


In [75]:
model_params.to_json()

{'temp': 1, 'max_tokens': 1000}

In [79]:
import json

with open(file = '../data/schema.json') as f:
    test_json = json.load(f)

In [83]:
test_json['chat_history'][0]['chat_interaction']

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'What is the weather like today?'},
 {'role': 'assistant',
  'content': 'It will be a nice day today.',
  'metadata': {'model_name': 'mistral_7b',
   'meta_prompt': 'You are a helpful assistant.',
   'temperature': 0.7,
   'max_tokens': 1000,
   'like_data': None}}]